In [10]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import os
import os
import sys
import pandas as pd
from typing import List, Dict, Optional
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")
import plotly.express as px

In [11]:
dataset_path = r'C:\Users\TPWODL\New folder_Content\AutonomousDataAnalystAgent\data\raw\x_data.xlsx'

In [14]:
main_df = pd.read_excel(r'C:\Users\TPWODL\New folder_Content\AutonomousDataAnalystAgent\data\raw\x_data.xlsx')

In [15]:
main_df.shape

(32873, 34)

In [25]:
main_df.columns

Index(['SL.NO', 'DATE', 'SHIFT DUTY', 'QUERY/REQUEST/COMPLAINT',
       'COMPLAINT DETAILS', 'COMPLAINT NUMBER', 'SECTION', 'SUB-DIVISION',
       'DIVISION', 'CIRCLE', 'COMPLAINT TYPE', 'CONSUMER NUMBER',
       'MOBILE NUMB', 'DEPT', 'CLOSED/OPEN', 'REMARKS', 'TWEET/LINK',
       'COMPLAINANT NAME', 'COMPLAINT RECEIVED TIME', 'RESPONSE TIME',
       'SECOND RESPONSE TIME', 'FINAL RESPONSE TIME',
       'FINAL RESPONSE DATE DD/MM/YYYY', 'PSCC/FG/TO', 'ARREARS',
       'REPEAT (Y / N)', 'FORWARDED TO', 'SENTIMENTS', 'NATURE OF TWEET',
       'ACTIONABLE/ NON ACTIONABLE', 'AGEING', 'SLAB', 'SLAB2', 'MINUTE'],
      dtype='object')

In [32]:
df = main_df.copy()

In [33]:
    # Ensure DATE column is datetime
df['DATE'] = pd.to_datetime(df['DATE'])

    # **FIX: Clean data before pivoting**
df['COMPLAINT TYPE'] = df['COMPLAINT TYPE'].astype(str).str.strip().str.title()
df['DEPT'] = df['DEPT'].astype(str).str.strip().str.title()
df['CLOSED/OPEN'] = df['CLOSED/OPEN'].astype(str).str.strip().str.title()
    # Calculate age in days
today = pd.Timestamp.today()
df['Age_Days'] = (today - df['DATE']).dt.days

    # Define age buckets (FIXED: bin ranges to match labels correctly)
bins = [0, 15, 30, 60, 90, 180, float('inf')]
labels = ['<15Days', '16-30Days', '31-60Days', '61-90Days', '91-180Days', '>180Days']
df['Age_Bucket'] = pd.cut(df['Age_Days'], bins=bins, labels=labels, right=True, include_lowest=True) 

In [34]:
# Pivot table with complaint type + age bucket + dept + open/close
pivot_data_df = pd.pivot_table(
    df,
    values='SL.NO',        
    index=['COMPLAINT TYPE'],              # rows
    columns=['Age_Bucket','DEPT','CLOSED/OPEN'],  # ✅ include CLOSED/OPEN in columns
    aggfunc='count',
    fill_value=0
)

# ➡️ Add Grand Total column (row-wise sum)
pivot_data_df['Grand_Total'] = pivot_data_df.sum(axis=1)

# ➡️ Add Grand Total row (column-wise sum)
pivot_data_df.loc['Grand_Total'] = pivot_data_df.sum(axis=0)


In [35]:
# Convert to dictionary (orient='records' for better format)
all_aging_dict = pivot_data_df.to_dict(orient='records')

In [36]:
all_aging_dict

[{('<15Days', 'Commercial', 'Closed'): 1,
  ('<15Days', 'Commercial', 'Open'): 0,
  ('<15Days', 'O&M', 'Closed'): 0,
  ('<15Days', 'O&M', 'Open'): 0,
  ('<15Days', 'Other', 'Closed'): 0,
  ('<15Days', 'Other', 'Open'): 0,
  ('16-30Days', 'Commercial', 'Closed'): 16,
  ('16-30Days', 'Commercial', 'Open'): 0,
  ('16-30Days', 'O&M', 'Closed'): 0,
  ('16-30Days', 'O&M', 'Open'): 0,
  ('16-30Days', 'Other', 'Closed'): 0,
  ('16-30Days', 'Other', 'Open'): 0,
  ('31-60Days', 'Commercial', 'Closed'): 23,
  ('31-60Days', 'Commercial', 'Open'): 1,
  ('31-60Days', 'O&M', 'Closed'): 0,
  ('31-60Days', 'O&M', 'Open'): 0,
  ('31-60Days', 'Other', 'Closed'): 0,
  ('31-60Days', 'Other', 'Open'): 0,
  ('61-90Days', 'Commercial', 'Closed'): 20,
  ('61-90Days', 'Commercial', 'Open'): 0,
  ('61-90Days', 'O&M', 'Closed'): 0,
  ('61-90Days', 'O&M', 'Open'): 0,
  ('61-90Days', 'Other', 'Closed'): 0,
  ('61-90Days', 'Other', 'Open'): 0,
  ('91-180Days', 'Commercial', 'Closed'): 53,
  ('91-180Days', 'Commercia

In [37]:
all_aging_df = pd.DataFrame(all_aging_dict)

In [38]:
all_aging_df 

,"(<15Days, Commercial, Closed)","(<15Days, Commercial, Open)","(<15Days, O&M, Closed)","(<15Days, O&M, Open)","(<15Days, Other, Closed)","(<15Days, Other, Open)","(16-30Days, Commercial, Closed)","(16-30Days, Commercial, Open)","(16-30Days, O&M, Closed)","(16-30Days, O&M, Open)",...,"(91-180Days, O&M, Open)","(91-180Days, Other, Closed)","(91-180Days, Other, Open)","(>180Days, Commercial, Closed)","(>180Days, Commercial, Open)","(>180Days, O&M, Closed)","(>180Days, O&M, Open)","(>180Days, Other, Closed)","(>180Days, Other, Open)","(Grand_Total, , )"
0,1,0,0,0,0,0,16,0,0,0,...,0,0,0,706,0,0,0,0,0,820
1,0,0,24,5,0,0,0,0,25,10,...,52,0,0,0,0,1478,48,0,0,1901
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,643,0,0,0,643
3,0,0,5,0,0,0,0,0,4,0,...,0,0,0,0,0,1090,0,0,0,1312
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,182,0,0,0,0,0,193
5,0,0,21,0,0,0,0,0,56,0,...,0,0,0,0,0,5074,0,0,0,6694
6,15,5,0,0,0,0,22,3,0,0,...,0,0,0,617,0,0,0,0,0,777
7,0,0,0,0,27,1,0,0,2,0,...,0,533,0,0,0,371,0,3662,1,4887
8,2,0,0,0,0,0,5,0,0,0,...,0,0,0,113,0,0,0,0,0,140
9,0,0,0,1,0,0,0,0,13,4,...,24,0,0,0,0,834,33,0,0,1044
